#### Before You Start


cd .devcontainer </br>
run docker-compose up

In [1]:
! pip install psycopg2
! pip install sqlalchemy



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import psycopg2  #import of the psycopg2 python library
import pandas as pd #import of the pandas python library
import pandas.io.sql as psql

##No transaction is started when commands are executed and no commit() or rollback() is required. 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [3]:
try:
    # Connect to the postgreSQL server with username, and password credentials
    con = psycopg2.connect(user = "postgres",
                                  password = "postgres",
                                  host = "localhost",
                                  port = "5432")
    
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    print("Connected Successfully to PostgreSQL server!!")
    
    # Obtain a DB Cursor to perform database operations
    cursor = con.cursor();
except (Exception, psycopg2.Error) as error :
     print ("Error while connecting to PostgreSQL", error)


Connected Successfully to PostgreSQL server!!


### Test that reading CSV works

In [4]:
import pandas as pd
df = pd.read_csv('circuits.csv')
print(df.head())  # Displays the first 5 rows


   circuit_id                            name      location    country  \
0           1  Albert Park Grand Prix Circuit     Melbourne  Australia   
1           2    Sepang International Circuit  Kuala Lumpur   Malaysia   
2           3   Bahrain International Circuit        Sakhir    Bahrain   
3           4  Circuit de Barcelona-Catalunya      Montmeló      Spain   
4           5                   Istanbul Park      Istanbul     Turkey   

        lat       long  alt                                                url  
0 -37.84970  144.96800   10  http://en.wikipedia.org/wiki/Melbourne_Grand_P...  
1   2.76083  101.73800   18  http://en.wikipedia.org/wiki/Sepang_Internatio...  
2  26.03250   50.51060    7  http://en.wikipedia.org/wiki/Bahrain_Internati...  
3  41.57000    2.26111  109  http://en.wikipedia.org/wiki/Circuit_de_Barcel...  
4  40.95170   29.40500  130         http://en.wikipedia.org/wiki/Istanbul_Park  


### Create sample table

In [14]:
#Create "test" Table

try:
    #table_name variable
    seasonTable="season"
    create_all_tables_query = '''
    CREATE TABLE IF NOT EXISTS circuit (
        circuit_id INT PRIMARY KEY,
        -- circuit_ref omitted
        name TEXT,
        location TEXT,
        country TEXT,
        lat float,
        long float,
        alt INT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS constructor (
        constructor_id INT PRIMARY KEY,
        -- removing constructor_ref for normalisation purposes
        name TEXT,
        nationality TEXT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS driver (
        driver_id INT PRIMARY KEY,
        -- removing driver_ref and number for normalisation purposes
        code TEXT,
        first_name TEXT, --no longer forename
        last_name TEXT, --no longer surname
        dob DATE,
        nationality TEXT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS lap_time (
        race_id INT PRIMARY KEY,
        driver_id INT REFERENCES driver,
        lap INT,
        position INT,
        time_in_milliseconds INT
    );

    CREATE TABLE IF NOT EXISTS pit_stop (
        race_id INT,
        driver_id INT,
        PRIMARY KEY (race_id, driver_id),
        stop INT,
        lap INT,
        time_in_miliseconds INT
    );

    CREATE TABLE IF NOT EXISTS race (
        race_id INT PRIMARY KEY,
        year INT,
        round INT,
        circuit_id INT REFERENCES circuit,
        name TEXT,
        date DATE,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS result (
        result_id INT PRIMARY KEY,
        race_id INT REFERENCES race,
        driver_id INT REFERENCES driver,
        constructor_id INT REFERENCES constructor,
        -- number and grid omitted
        position INT, -- use positionOrder instead of position/positionText
        points INT,
        laps INT,
        -- omit time, unclean
        time_in_miliseconds INT,
        fastestLap INT,
        rank INT,
        fastest_lap_time INT,
        fastest_lap_speed float
    );

    CREATE TABLE IF NOT EXISTS season (
        year INT PRIMARY KEY,
        url TEXT
    );

    '''


    #Execute this command (SQL Query)
    cursor.execute(create_all_tables_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("All tables created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print("Error While Creating the DB: ",error)

All tables created successfully in PostgreSQL 


Check table creation

In [6]:
# [information_schema.tables] keep listing of every table being managed by Postgres for a particular database.
# specifying the tabel_schema to 'public' to only list tables that you create.
cursor.execute("""SELECT table_name 
                  FROM information_schema.tables 
                  WHERE table_schema = 'public'  
               """)

for table in cursor.fetchall():
    print(table)

('season',)
('result',)
('racestest',)
('circuit',)
('race',)
('constructor_result',)
('constructor',)
('constructor_standing',)
('driver_standing',)
('driver',)
('lap_time',)
('pit_stop',)
('result_id',)


### Read seasons.CSV and insert into already created tables with constraints

In [8]:
# import pandas as pd
# from sqlalchemy import create_engine

# # Load the CSV file into a pandas DataFrame
# csv_file_path = "seasons.csv"  # Replace with your CSV file path
# table_name = "season"  # Replace with your PostgreSQL table name
# df = pd.read_csv(csv_file_path)

# # Connect to the PostgreSQL database
# # Replace the placeholders with your database credentials
# user = "postgres"
# password = "postgres"
# host = "localhost"
# port = "5432"
# database = "postgres"
# engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# # Loop through the list of CSV files and import them into the PostgreSQL tables
# for csv_file, table_name in csv_files:
#     try:
#         # Load CSV file into DataFrame
#         df = pd.read_csv(csv_file)

#         # Write the DataFrame to the PostgreSQL table
#         df.to_sql(table_name, con=engine, if_exists="append", index=False)
#         print(f"Data from {csv_file} successfully imported into {table_name}!")
#     except Exception as e:
#         print(f"An error occurred while importing {csv_file} into {table_name}: {e}")



In [9]:
# # Load the CSV file into a pandas DataFrame
# csv_file_path = "circuits.csv"  # Replace with your CSV file path
# table_name = "circuit"  # Replace with your PostgreSQL table name
# df = pd.read_csv(csv_file_path)

# # Rename DataFrame columns to match the PostgreSQL table schema
# df.rename(columns={
#     "circuitId": "circuit_id",
#     "lng": "long",
#     # Add any other necessary mappings here
# }, inplace=True)

# # Write the DataFrame to the PostgreSQL table
# try:
#     df.to_sql(table_name, con=engine, if_exists="append", index=False)
#     print(f"Data from {csv_file_path} successfully imported into {table_name}!")
# except Exception as e:
#     print(f"An error occurred: {e}")


### Read all CSVs and insert into postgres

In [15]:
import pandas as pd
from sqlalchemy import create_engine

# List of CSV files and corresponding table names
csv_files = [
    ("circuit.csv", "circuit"),
    ("constructor.csv", "constructor"),
    ("driver.csv", "driver"),
    ("lap_time.csv", "lap_time"),
    ("pit_stop.csv", "pit_stop"),
    ("race.csv", "race"),
    ("result.csv", "result"),
    ("season.csv", "season")
]

# Connect to the PostgreSQL database
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Loop through the list of CSV files and import them into the PostgreSQL tables
for csv_file, table_name in csv_files:
    try:
        # Load CSV file into DataFrame
        df = pd.read_csv(csv_file)

        # Write the DataFrame to the PostgreSQL table
        df.to_sql(table_name, con=engine, if_exists="append", index=False)
        print(f"Data from {csv_file} successfully imported into {table_name}!")
    except Exception as e:
        print(f"An error occurred while importing {csv_file} into {table_name}: {e}")


# import pandas as pd
# from sqlalchemy import create_engine

# # Load the CSV file into a pandas DataFrame
# csv_file_path = "races.csv"  # Replace with your CSV file path
# table_name = "race"  # Replace with your PostgreSQL table name
# df = pd.read_csv(csv_file_path)

# # Rename DataFrame columns to match the PostgreSQL table schema
# df.rename(columns={
#     "raceId": "race_id",
#     "circuitId": "circuit_id",
#     # Add any other necessary mappings here
# }, inplace=True)

# columns_to_keep = ["race_id", "year", "round", "circuit_id", "name", "date", "url"]
# df = df[columns_to_keep]

# # Connect to the PostgreSQL database
# # Replace the placeholders with your database credentials
# user = "postgres"
# password = "postgres"
# host = "localhost"
# port = "5432"
# database = "postgres"
# engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# # Write the DataFrame to the PostgreSQL table
# try:
#     df.to_sql(table_name, con=engine, if_exists="append", index=False)
#     print(f"Data from {csv_file_path} successfully imported into {table_name}!")
# except Exception as e:
#     print(f"An error occurred: {e}")


An error occurred while importing circuit.csv into circuit: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "circuit_pkey"
DETAIL:  Key (circuit_id)=(1) already exists.

[SQL: INSERT INTO circuit (circuit_id, name, location, country, lat, long, alt, url) VALUES (%(circuit_id__0)s, %(name__0)s, %(location__0)s, %(country__0)s, %(lat__0)s, %(long__0)s, %(alt__0)s, %(url__0)s), (%(circuit_id__1)s, %(name__1)s, %(location__1)s ... 9202 characters truncated ... (name__76)s, %(location__76)s, %(country__76)s, %(lat__76)s, %(long__76)s, %(alt__76)s, %(url__76)s)]
[parameters: {'circuit_id__0': 1, 'country__0': 'Australia', 'long__0': 144.968, 'url__0': 'http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit', 'name__0': 'Albert Park Grand Prix Circuit', 'alt__0': 10, 'location__0': 'Melbourne', 'lat__0': -37.8497, 'circuit_id__1': 2, 'country__1': 'Malaysia', 'long__1': 101.738, 'url__1': 'http://en.wikipedia.org/wiki/Sepang_International_Circuit', 'name__1

### View query

In [11]:
# sql_select_query = """ SELECT * FROM season WHERE year = 2001"""
# sql_select_query = """ SELECT r.race_id, r.year FROM race as r JOIN season as s ON s.year = r.year WHERE s.year = 2001"""

# try:
#     cursor.execute(sql_select_query, (1,))
#     person_records = cursor.fetchall() 
#     print("Print each row and it's columns values:\n")
#     # print(person_records)
#     df = pd.DataFrame(person_records, columns=['race_id', 'year'])
#     print(df)
# except(Exception, psycopg2.Error) as error :
#     con.rollback()
#     print("Error:", error)


Countries_Customers_Cnt_gt1= psql.read_sql("""SELECT r.race_id, r.year 
                                           FROM race as r 
                                           JOIN season as s 
                                           ON s.year = r.year 
                                           WHERE s.year = 2001
                                          """, engine)
display(Countries_Customers_Cnt_gt1.style)

,race_id,year
0,141,2001
1,142,2001
2,143,2001
3,144,2001
4,145,2001
5,146,2001
6,147,2001
7,148,2001
8,149,2001
9,150,2001
